# Rag_QA_with_Gemini
제작 : 박광석(모두의연구소, https://www.linkedin.com/in/andkspark)  
  
해당 노트는 Langchain으로 구현된 Question-Answer RAG 완성 예제입니다

참고 :
https://python.langchain.com/docs/get_started/introduction  
https://python.langchain.com/docs/integrations/llms/google_ai/  
YouTube <모두의AI> , <테디노트>   
인프런 <모두를 위한 대규모 언어 모델 LLM(Large Language Model) Part 2 - 랭체인(LangChain)으로 나만의 ChatGPT 만들기>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
YOUR_API_KEY = ''

필요한 라이브러리를 모두 다운받습니다  

In [ ]:
!pip install -q langchain langchain-google-genai chromadb pypdf sentence_transformers tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 32.4 MB/s eta 0:00:0

In [ ]:
!pip install U -q langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00


In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = YOUR_API_KEY

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

Text splitter 사용을 위한 준비입니다

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)

    return len(tokens)

### Step 1 Document loader

In [ ]:
loader = PyPDFLoader("/content/Demian.pdf")
pages = loader.load_and_split()



### Step 2 Text splitters

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=50, length_function = tiktoken_len)
texts = text_splitter.split_documents(pages)

### Step 3 Vector Empeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-10-235270254412>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
docsearch = Chroma.from_documents(texts, embedding_model)

### Step 4 Retrievers

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# QA

llm_gemini = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temperature=0.0)

In [ ]:
qa = RetrievalQA.from_chain_type(llm_gemini, chain_type="stuff",
                                 retriever=docsearch.as_retriever(
                                     search_type="mmr",
                                     search_kwargs={"k": 3, "fetch_k" : 10 }
                                 ))

### Question Answering

In [ ]:
query = "how demian looks like"
result = qa(query)

In [ ]:
from IPython.display import Markdown, display
display(Markdown(result["result"]))

**Physical Appearance:**

* **Age:** 15-16 years old
* **Height:** Average
* **Build:** Slender and athletic
* **Hair:** Dark brown, short and neatly combed
* **Eyes:** Deep blue, piercing and expressive
* **Facial Features:** Sharp and angular, with a strong jawline and high cheekbones
* **Skin:** Fair and smooth

**Clothing:**

* **School Uniform:** Black blazer, white shirt, gray trousers, and a black tie
* **Casual Wear:** Jeans, T-shirts, hoodies, and sneakers

**Other Notable Features:**

* **Scar:** A small scar on his left cheekbone, a reminder of a childhood accident
* **Brooch:** A silver brooch in the shape of a raven, a symbol of his connection to the supernatural
* **Aura:** An enigmatic and slightly unsettling presence, as if he carries a secret within him

**Personality:**

* **Intelligent:** Quick-witted and perceptive, with a thirst for knowledge
* **Introspective:** Deeply introspective and self-aware, often lost in his own thoughts
* **Rebellious:** Defiant and questioning of authority, seeking to break free from societal norms
* **Sensitive:** Highly sensitive and empathetic, able to connect with others on a deep level
* **Mysterious:** Enigmatic and secretive, keeping his true thoughts and feelings hidden from most

수고하셨습니다!